In [350]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
os.chdir("C:/Users/x01382250/Documents/Backtesteur Anthony")

** Import the data **

In [351]:
df_data = pd.read_excel("Backtester - Dispersion.xlsm", sheet_name = "Data", skiprows = 1, index_col= None).dropna(axis=1, how="all")
df_control = pd.read_excel("Backtester - Dispersion.xlsm",skiprows=6,sheet_name = "Control", usecols= "G,I,K,M,O,Q,S,U")

data_index = df_data.iloc[:,[1]].set_index(np.array(df_data.iloc[:,0]))
data_index = data_index.loc[data_index.index.notnull()]
N=df_control.iloc[2,3]
strike=np.array(df_control.iloc[2:,4])
start_date = df_control.iloc[2,6]
strike_index = df_control.iloc[2,5]
weights = np.array(df_control.iloc[1:,1].dropna(axis = 0))

** Create a dictionary with tickers and their weights (alphabetical order) **

In [352]:
weights = np.array(df_control.iloc[1:,1].dropna(axis = 0))
tickers_name =  np.array(df_control.iloc[1:,0].dropna(axis = 0))
dico={}
for key,weight in zip (tickers_name, weights):
    dico[key]=weight
dic_weights={}
dic_weights = {dico[key] for key in sorted(dico)}
sorted_dict = {key: value for key, value in sorted(dico.items())}
weights = np.array(list(sorted_dict.values()))

#### We create a class "Vol Swap" containing all the data
* `__init__` : loads the parameters
* `price vectors` : 
    * from the data vector, we compute a vector containing all the backtest values from start date to today.
    * compute the price; for each stock, it is the following payoff:
    
   <center> $P = FRV \wedge 2.5K - K$     &nbsp; where&nbsp;&nbsp;$FRV=\sqrt{\frac{252}{N}\sum_{t=1}^{N}{ln(\frac{S_t}{S_{t-1}})}^2}$  



In [353]:
class vol_swap:
    def __init__(self,N,strike,data):
        self.N = N
        self.K = strike
        self.data = data 
        self.price_vector()
    def price_vector(self):
        ret  = self.data.pct_change()**2
        ret.iloc[:,:1] =np.sqrt((ret.iloc[:,:1]).rolling(N).mean()*252)
        ret.iloc[:,:1]=(ret.iloc[:,:1].clip(upper=self.K*2.5)-self.K )*100
        self.prices = ret

#### Compute the prices for the index and the stocks
* initialize the dataframe with the index values
* create a `vol_swap` object for each underlying to get the prices
* merge the dataframes in order to get all calendar dates
* remove the N first rows as they correspond to null backtest ($N_{obs} < N$)

In [354]:
def compute_backtest():
    backtest = df_data.iloc[:,:2]
    backtest=backtest.set_index(np.array(backtest.iloc[:,0])).iloc[:,:1]
    for i in range(1,int(len(df_data.columns)/2)):  
        data = df_data.iloc[:, [2*i,2*i+1]]
        data = data.set_index(np.array(data.iloc[:,0]))
        data = data.iloc[:,1:]
        swap_stock = vol_swap(N,strike[i-1],data)
        swap_stock.prices  = swap_stock.prices[swap_stock.prices.index.notnull()]
        if i==1:
            backtest= swap_stock.prices.iloc[N:,:]
        else :
            backtest = pd.merge(backtest, swap_stock.prices.iloc[N:,:], left_index=True, right_index=True)
    backtest = backtest.reindex(sorted(backtest.columns), axis=1)
    return backtest
backtest = compute_backtest()

#### Compute the price, and reweight when necessary


* replace "NA" by 0 to ease computations (days where no stock is available)

In [355]:
backtest=backtest.fillna(0)

* compute the weighted sum of the prices


In [356]:
backtest["weighted_sum"] = (weights.T*backtest).sum(axis=1)

* compute the sum of the weights: if the day is not available, swap does not enter in the computations.


In [357]:
df_is_na = backtest.iloc[:,:-1]!=0
backtest["sum_of_weights"] =(weights.T*df_is_na).sum(axis = 1)

* compute the weighted average : arithmetical or geometric


In [358]:
backtest["result_stocks"] = backtest["weighted_sum"]/backtest["sum_of_weights"]

* compute data for the index

In [359]:
index_swap = vol_swap(N,strike_index,data_index)
backtest =pd.merge(backtest,index_swap.prices.iloc[N:,:],left_index=True,right_index=True) 

* compute dispersion prices : long stock swaps, short index swap

In [360]:
backtest["result"] = backtest["result_stocks"] -  backtest.iloc[:,-1]

## We repeat the exact same for a 60 days calendar

In [361]:
N=60
backtest_60 = compute_backtest()
backtest_60=backtest_60.fillna(0)
backtest_60["weighted_sum"] = (weights.T*backtest_60).sum(axis=1)
df_is_na_60 = backtest_60.iloc[:,:-1]!=0
backtest_60["sum_of_weights"] =(weights.T*df_is_na_60).sum(axis = 1)
backtest_60["result_stocks"] = backtest_60["weighted_sum"]/backtest_60["sum_of_weights"]
index_swap_60 = vol_swap(N,strike_index,data_index)
backtest_60 =pd.merge(backtest_60,index_swap_60.prices.iloc[N:,:],left_index=True,right_index=True) 
backtest_60["result"] = backtest_60["result_stocks"] -  backtest_60.iloc[:,-1]

# Compute Hit Ratios, means,...

In [362]:
def compute_dataframes_ratio(backtest,start,end):
    backtest_compute = backtest.drop(["weighted_sum","sum_of_weights","result_stocks","result"], axis = 1).iloc[backtest.index.get_loc(start, method='nearest'):backtest.index.get_loc(end, method='nearest')+1,:]
    backtest_compute = backtest_compute.sub(backtest_compute.iloc[:,-1],axis=0).iloc[:,:-1]
    results = pd.DataFrame(columns = backtest_compute.columns)
    results.loc["Mean"] = backtest_compute.mean()
    results.loc["Min"] = backtest_compute.min()
    results.loc["Max"] = backtest_compute.max()
    results.loc["Last"] = backtest_compute.iloc[-1,:]
    results.loc["Hit Ratio"] = (backtest_compute>0).sum()/len(backtest.index)
    results.loc["Hit Ratio"]=((results.loc['Hit Ratio']).apply('{:.00%}'.format))
    return results

#### Compute data for range period and no range

In [363]:
start = df_control.iloc[2,7]
end = df_control.iloc[3,7]
results_range = compute_dataframes_ratio(backtest,start,end)
results_no_range = compute_dataframes_ratio(backtest,start_date,backtest.index[-1])

#### Add 60 days data to the two dataframes

In [364]:
results_range.loc["Last (60 days)"] = backtest_60.iloc[backtest_60.index.get_loc(end, method='nearest'),:]
results_no_range.loc["Last (60 days)"]  = backtest_60.iloc[-1,:]

# Export to excel and create a chart

In [365]:
excel_file = 'Backtest - Results.xlsx'
sheet_name = 'Prices'
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter',datetime_format='dd/mm/yyyy')
backtest.to_excel(writer, sheet_name=sheet_name)
backtest_60.to_excel(writer,sheet_name="Prices 60")

results_no_range.to_excel(writer,sheet_name="Mono")
results_range.to_excel(writer,sheet_name="Mono",startrow = 15)


workbook = writer.book
fmt_bleu= workbook.add_format({"bg_color": "#DAEEF3"})
worksheet= workbook.add_worksheet("Graph")
worksheet.set_column('A:ZZ', None, fmt_bleu)

percentage = workbook.add_format({'num_format': '0.00%'})



chart1 = workbook.add_chart({'type': 'line'})

chart1.add_series({
    'categories': ['Prices', 1, 0, len(backtest.index), 0],
    'values':     ['Prices', 1, len(backtest.columns), len(backtest.index), len(backtest.columns)],
})

chart1.set_plotarea({
    'fill':   {'color': '#E9F1F5'},
    'line':   {'color' : "#46AAC5"}
})


chart1.set_title ({'name': 'Backtest'})
chart1.set_x_axis({
    'name': 'Dates',
    'visible': True,
    'line': {'width': 1.25, 'dash_type': 'dash'}, 
    "date_axis": True, 
    'min':backtest.index[0] ,
    'max': backtest.index[len(backtest.index)-1],})

chart1.set_y_axis({'name': 'P&L in Vol points'})
chart1.set_legend({'none': True})
worksheet.insert_chart('D2', chart1, {'x_offset': 25, 'y_offset': 10,'x_scale': 2, 'y_scale': 2})

worksheet60= workbook.add_worksheet("Graph 60")
worksheet60.set_column('A:ZZ', None, fmt_bleu)
chart2 = workbook.add_chart({'type': 'line'})


chart2 = workbook.add_chart({'type': 'line'})

chart2.add_series({
    'categories': ['Prices 60', 1, 0, len(backtest_60.index), 0],
    'values':     ['Prices 60', 1, len(backtest_60.columns), len(backtest_60.index), len(backtest_60.columns)],
})

chart2.set_plotarea({
    'fill':   {'color': '#E9F1F5'},
    'line':   {'color' : "#46AAC5"}
})


chart2.set_title ({'name': 'Carry 3 Months'})
chart2.set_x_axis({
    'name': 'Dates',
    'visible': True,
    'line': {'width': 1.25, 'dash_type': 'dash'}, 
    "date_axis": True, 
    'min':backtest_60.index[0] ,
    'max': backtest_60.index[len(backtest_60.index)-1],})

chart2.set_y_axis({'name': 'P&L in Vol points'})
chart2.set_legend({'none': True})
worksheet60.insert_chart('D2', chart2, {'x_offset': 25, 'y_offset': 10,'x_scale': 2, 'y_scale': 2})


worksheet.write(0,0,"Min")
worksheet.write(1,0,"Max")
worksheet.write(2,0,"Mean")
worksheet.write(3,0,"Std")
worksheet.write(0,1,backtest["result"].min())
worksheet.write(1,1,backtest["result"].max())
worksheet.write(2,1,backtest["result"].mean())
worksheet.write(3,1,backtest["result"].std())


worksheet60.write(0,0,"Min")
worksheet60.write(1,0,"Max")
worksheet60.write(2,0,"Mean")
worksheet60.write(3,0,"Std")
worksheet60.write(0,1,backtest_60["result"].min())
worksheet60.write(1,1,backtest_60["result"].max())
worksheet60.write(2,1,backtest_60["result"].mean())
worksheet60.write(3,1,backtest_60["result"].std())

writer.save()